In [1]:
# imports
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

print("Size of:")
print("- Training-set:\t\t{}".format(len(mnist.train.labels)))
print("- Test-set:\t\t{}".format(len(mnist.test.labels)))
print("- Validation-set:\t{}".format(len(mnist.validation.labels)))

# hyper-parameters
logs_path = "./logs/jupyter/embedding/"  # path to the folder that we want to save the logs for Tensorboard
learning_rate = 0.005  # The optimization learning rate
epochs = 5  # Total number of training epochs
batch_size = 100  # Training batch size
display_freq = 100  # Frequency of displaying the training results

# Network Parameters
# We know that MNIST images are 28 pixels in each dimension.
img_h = img_w = 28

# Images are stored in one-dimensional arrays of this length.
img_size_flat = img_h * img_w

# Number of classes, one class for each of 10 digits.
n_classes = 10

# number of units in the first hidden layer
h1 = 200

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [2]:
# weight and bais wrappers
def weight_variable(name, shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W_' + name,
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(name, shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b_' + name,
                           dtype=tf.float32,
                           initializer=initial)

def fc_layer(x, num_units, name, use_relu=True):
    """
    Create a fully-connected layer
    :param x: input from previous layer
    :param num_units: number of hidden units in the fully-connected layer
    :param name: layer name
    :param use_relu: boolean to add ReLU non-linearity (or not)
    :return: The output array
    """
    with tf.variable_scope(name):
        in_dim = x.get_shape()[1]
        W = weight_variable(name, shape=[in_dim, num_units])
        tf.summary.histogram('W', W)
        b = bias_variable(name, [num_units])
        tf.summary.histogram('b', b)
        layer = tf.matmul(x, W)
        layer += b
        if use_relu:
            layer = tf.nn.relu(layer)
        return layer



def write_sprite_image(filename, images):
    """
        Create a sprite image consisting of sample images
        :param filename: name of the file to save on disk
        :param shape: tensor of flattened images
    """

    # Invert grayscale image
    images = 1 - images

    # Calculate number of plot
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))

    # Make the background of sprite image
    sprite_image = np.ones((img_h * n_plots, img_w * n_plots))

    for i in range(n_plots):
        for j in range(n_plots):
            img_idx = i * n_plots + j
            if img_idx < images.shape[0]:
                img = images[img_idx]
                sprite_image[i * img_h:(i + 1) * img_h,
                j * img_w:(j + 1) * img_w] = img

    plt.imsave(filename, sprite_image, cmap='gray')
    print('Sprite image saved in {}'.format(filename))

def write_metadata(filename, labels):
    """
            Create a metadata file image consisting of sample indices and labels
            :param filename: name of the file to save on disk
            :param shape: tensor of labels
    """
    with open(filename, 'w') as f:
        f.write("Index\tLabel\n")
        for index, label in enumerate(labels):
            f.write("{}\t{}\n".format(index, label))

    print('Metadata file saved in {}'.format(filename))

In [29]:
# Load the data set
X_test = mnist.test.images
Y_test = mnist.test.labels

X_train = mnist.train.images
Y_train = mnist.train.labels

X_val = mnist.validation.images
Y_val = mnist.validation.labels

In [4]:


g_1=tf.Graph()
with g_1.as_default():
    # Create graph
    # Placeholders for inputs (x), outputs(y)
    with tf.variable_scope('Input'):
        x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='X')
        tf.summary.image('input_image', tf.reshape(x, (-1, img_w, img_h, 1)), max_outputs=5)
        y = tf.placeholder(tf.float32, shape=[None, n_classes], name='Y')
    #fc1 = fc_layer(x, h1, 'Hidden_layer', use_relu=True)
    output_logits = fc_layer(x, n_classes, 'Output_layer', use_relu=False)

    # Define the loss function, optimizer, and accuracy
    with tf.variable_scope('Train'):
        with tf.variable_scope('Loss'):
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_logits), name='loss')
            tf.summary.scalar('loss', loss)
        with tf.variable_scope('Optimizer'):
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op').minimize(loss)
        with tf.variable_scope('Accuracy'):
            correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1), name='correct_pred')
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
            tf.summary.scalar('accuracy', accuracy)
            # Network predictions
            cls_prediction = tf.argmax(output_logits, axis=1, name='predictions')

    # Initializing the variables
    init = tf.global_variables_initializer()
    merged = tf.summary.merge_all()

    # Initialize the embedding variable with the shape of our desired tensor
    tensor_shape = (X_test.shape[0] , output_logits.get_shape()[1].value) # [test_set , h1] = [10000 , 200]
    embedding_var = tf.Variable(tf.zeros(tensor_shape), name='fc1_embedding')
    # assign the tensor that we want to visualize to the embedding variable
    embedding_assign = embedding_var.assign(output_logits)

In [7]:
# Launch the graph (session)
#sess = tf.InteractiveSession() # using InteractiveSession instead of Session to test network in separate cell
with tf.Session(graph=g_1) as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(logs_path, sess.graph)
    num_tr_iter = int(mnist.train.num_examples / batch_size)
    global_step = 0
    for epoch in range(epochs):
        print('Training epoch: {}'.format(epoch + 1))
        for iteration in range(num_tr_iter):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            global_step += 1
            # Run optimization op (backprop)
            feed_dict_batch = {x: batch_x, y: batch_y}
            _, summary_tr = sess.run([optimizer, merged], feed_dict=feed_dict_batch)
            train_writer.add_summary(summary_tr, global_step)

            if iteration % display_freq == 0:
                # Calculate and display the batch loss and accuracy
                loss_batch, acc_batch = sess.run([loss, accuracy],feed_dict=feed_dict_batch)
                print("iter {0:3d}:\t Loss={1:.2f},\tTraining Accuracy={2:.01%}".format(iteration, loss_batch, acc_batch))

        # Run validation after every epoch
        feed_dict_valid = {x: mnist.validation.images, y: mnist.validation.labels}
        loss_valid, acc_valid = sess.run([loss, accuracy], feed_dict=feed_dict_valid)
        print('---------------------------------------------------------')
        print("Epoch: {0}, validation loss: {1:.2f}, validation accuracy: {2:.01%}".format(epoch + 1, loss_valid, acc_valid))
        print('---------------------------------------------------------')

    from tensorflow.contrib.tensorboard.plugins import projector

    # Create a config object to write the configuration parameters
    config = projector.ProjectorConfig()

    # Add embedding variable
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name

    # Link this tensor to its metadata file (e.g. labels) -> we will create this file later
    embedding.metadata_path = 'metadata.tsv'

    # Specify where you find the sprite. -> we will create this image later
    embedding.sprite.image_path = 'sprite_images.png'
    embedding.sprite.single_image_dim.extend([img_w, img_h])

    # Write a projector_config.pbtxt in the logs_path.
    # TensorBoard will read this file during startup.
    projector.visualize_embeddings(train_writer, config)

    # Run session to evaluate the tensor
    x_test_fc1 = sess.run(embedding_assign, feed_dict={x: X_test})

    # Save the tensor in model.ckpt file
    saver = tf.train.Saver()
    saver.save(sess, os.path.join(logs_path, "model.ckpt"), global_step)


    # Reshape images from vector to matrix
    x_test_images = np.reshape(np.array(X_test), (-1, img_w, img_h))
    # Reshape labels from one-hot-encode to index
    x_test_labels = np.argmax(Y_test, axis=1)

    write_sprite_image(os.path.join(logs_path, 'sprite_images.png'), x_test_images)
    write_metadata(os.path.join(logs_path, 'metadata.tsv'), x_test_labels)

    # close the session after you are done with testing
    #sess.close()

Training epoch: 1
iter   0:	 Loss=2.02,	Training Accuracy=55.0%
iter 100:	 Loss=0.33,	Training Accuracy=91.0%
iter 200:	 Loss=0.34,	Training Accuracy=90.0%
iter 300:	 Loss=0.24,	Training Accuracy=95.0%
iter 400:	 Loss=0.24,	Training Accuracy=97.0%
iter 500:	 Loss=0.29,	Training Accuracy=92.0%
---------------------------------------------------------
Epoch: 1, validation loss: 0.29, validation accuracy: 92.3%
---------------------------------------------------------
Training epoch: 2
iter   0:	 Loss=0.25,	Training Accuracy=93.0%
iter 100:	 Loss=0.23,	Training Accuracy=96.0%
iter 200:	 Loss=0.30,	Training Accuracy=92.0%
iter 300:	 Loss=0.34,	Training Accuracy=93.0%
iter 400:	 Loss=0.43,	Training Accuracy=86.0%
iter 500:	 Loss=0.25,	Training Accuracy=94.0%
---------------------------------------------------------
Epoch: 2, validation loss: 0.27, validation accuracy: 92.3%
---------------------------------------------------------
Training epoch: 3
iter   0:	 Loss=0.27,	Training Accuracy=93

In [39]:
g_train=tf.Graph()
with g_train.as_default():

    with tf.name_scope("inputs"):
        X = tf.placeholder(tf.float32, [None, 784],name='x_data')
        tf.summary.image('input_image', tf.reshape(X, (-1, 28, 28, 1)), max_outputs=5)
        Ydata = tf.placeholder(tf.float32, [None, 10],name='y_data') 
    
    with tf.name_scope("fitting_variables"):
        with tf.name_scope("weights"):
            #initer = tf.truncated_normal_initializer(stddev=0.01)
            W = tf.Variable(tf.zeros([784,10]),name='w')
            tf.summary.histogram("wieghts",W)
        with tf.name_scope("bias"):
            b = tf.Variable(tf.ones([10]),name='b')
            tf.summary.histogram("biases",b)
    
    with tf.name_scope("operations"):
        prod=tf.matmul(X,W,name="mult") #transponemos W^t X^t ==> X W. 
        # cada dato es una fila en X, la matriz W la renombramos como W^t para tener W al final
        sumation=prod+b #sumar b verticalmente con broadcasting
        
    with tf.name_scope("outputs"):
        Ymod = tf.nn.softmax(sumation,name="Ymod") 
    
    with tf.name_scope("evaluation"):
        with tf.name_scope("cross_entropy"):
            cross_entropy = tf.reduce_mean(-tf.reduce_sum(Ydata * tf.log(Ymod+1e-8), reduction_indices=[1])) 
            tf.summary.scalar("cross_entropy",cross_entropy)
        
        with tf.name_scope("correct_prediction"):
            correct_prediction = tf.equal(tf.argmax(Ymod,axis=1), tf.argmax(Ydata,axis=1)) 
            #tf.summary.tensor_summary("correct_prediction",correct_prediction)
        
        with tf.name_scope("accuracy"):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 
            tf.summary.scalar("accuracy",accuracy)
        
    with tf.name_scope("optimizer"):
        train_step = tf.train.AdamOptimizer(learning_rate=0.01, name='Adam-op').minimize(cross_entropy)
        
    init=tf.global_variables_initializer()
    merged=tf.summary.merge_all()
    
    # Initialize the embedding variable with the shape of our desired tensor
    tensor_shape = (X_test.shape[0] , Ymod.get_shape()[1].value) # [test_set , h1] = [10000 , 200]
    embedding_var = tf.Variable(tf.zeros(tensor_shape), name='ymod_embedding')
    # assign the tensor that we want to visualize to the embedding variable
    embedding_assign = embedding_var.assign(Ymod)
        
import time

batch_size=100
no_of_epochs=20

iteration_loss=[]
iteration_acc=[]

#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

with tf.Session(graph=g_train) as sess1:
    
    summaries_dir="logs/myjupyter"
    time_stamp=time.strftime("%H:%M:%S") #string con formato hora:minuto:segundo
    summaries_dir=summaries_dir+time_stamp
    
    #directorios para guardar logs de summary, se utilizan en tensorboard
    train_writer = tf.summary.FileWriter(summaries_dir+'/train',sess1.graph)
    valid_writer = tf.summary.FileWriter(summaries_dir + '/valid',sess1.graph)
    test_writer = tf.summary.FileWriter(summaries_dir + '/test',sess1.graph)

    init.run()
    
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    
    #calculo inicial de accuracy, perdida,etc.
    summary,loss=sess1.run([merged,cross_entropy],feed_dict={X: X_train,Ydata: Y_train},
                            options=run_options,run_metadata=run_metadata) #adds metadada to summary 
    train_writer.add_summary(summary, 0)
    train_writer.add_run_metadata(run_metadata, 'loss_epoch%d' % 0)
    #print("Epoch:=",0,"; \t Epoch Loss:=",loss)
    
    summary,t_acc=sess1.run([merged,accuracy], feed_dict={X: X_train, Ydata: Y_train},
                             options=run_options,run_metadata=run_metadata) 
    train_writer.add_summary(summary, 0)
    train_writer.add_run_metadata(run_metadata, 'train_epoch%d' % 0)
    #print("Training Accuracy is", t_acc*100,"%")
    
    summary,v_acc=sess1.run([merged,accuracy], feed_dict={X: X_val,Ydata:Y_val},
                             options=run_options,run_metadata=run_metadata) #adds metadada to summary 
    valid_writer.add_summary(summary,0)
    valid_writer.add_run_metadata(run_metadata, 'valid_epoch%d' % 0)
    #print("Validation Accuracy is", v_acc*100,"%")
    
    summary,f_acc=sess1.run([merged,accuracy], feed_dict={X: X_test,Ydata:Y_test},
                             options=run_options,run_metadata=run_metadata) #adds metadada to summary 
    test_writer.add_summary(summary,0)
    test_writer.add_run_metadata(run_metadata, 'test_epoch%d' % 0)
    #print("Test Accuracy is", f_acc*100,"%")
    
    sum_data=np.array([0,loss,t_acc,v_acc,f_acc]) #agregar los datos de la epoca 0 en una lista
    
    
    
    for epoch in range(no_of_epochs):
        epoch_loss=0
        
        for i in range(int(mnist.train.num_examples/batch_size)): #una epoca se divide según el tamaño del batch 
            run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata = tf.RunMetadata()
            
            batch_xs, batch_ys = mnist.train.next_batch(batch_size) 
            train,loss,acc=sess1.run([train_step,cross_entropy,accuracy], feed_dict={X: batch_xs, Ydata: batch_ys},
                                     options=run_options,run_metadata=run_metadata)
            
            if i==0:
                train_writer.add_run_metadata(run_metadata, 'train_step%d' % (epoch))
            #calcular gradiente y pérdida sobre los datos del batch (se usan todas la variables de la gráfica)
            
            epoch_loss+=loss #calcula la pérdida por cada batch y la suma para tener la pérdida final en la época
            iteration_loss.append(loss)
            iteration_acc.append(acc)
        
        #print("Epoch:=",(epoch+1),"; \t Epoch Loss:=",epoch_loss)
        
        
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        
        summary,t_acc=sess1.run([merged,accuracy], feed_dict={X: X_train, Ydata: Y_train}, #calculates training accuracy 
                               options=run_options,run_metadata=run_metadata) #adds metadada to summary       
        train_writer.add_summary(summary, (epoch+1))
        train_writer.add_run_metadata(run_metadata, 'train_epoch%d' % (epoch+1))
        #print("Training Accuracy is", t_acc*100,"%")
        
        summary,v_acc=sess1.run([merged,accuracy], feed_dict={X: X_val,Ydata:Y_val},
                               options=run_options,run_metadata=run_metadata)
        valid_writer.add_summary(summary, (epoch+1))
        valid_writer.add_run_metadata(run_metadata, 'valid_epoch%d' % (epoch+1))
        #print("Validation Accuracy is", v_acc*100,"%")
        
        summary,f_acc=sess1.run([merged,accuracy], feed_dict={X: X_test,Ydata:Y_test},
                                options=run_options,run_metadata=run_metadata)
        test_writer.add_summary(summary, (epoch+1))
        test_writer.add_run_metadata(run_metadata, 'test_epoch%d' % (epoch+1))
        #print("Test Accuracy is", f_acc*100,"%")
        
        sum_data=np.vstack((sum_data,np.array([(epoch+1),epoch_loss,t_acc,v_acc,f_acc])))
        print("Finished Epoch",(epoch+1))
        
    from tensorflow.contrib.tensorboard.plugins import projector

    # Create a config object to write the configuration parameters
    config = projector.ProjectorConfig()

    # Add embedding variable
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name

    # Link this tensor to its metadata file (e.g. labels) -> we will create this file later
    embedding.metadata_path = 'metadata.tsv'

    # Specify where you find the sprite. -> we will create this image later
    embedding.sprite.image_path = 'sprite_images.png'
    embedding.sprite.single_image_dim.extend([img_w, img_h])

    # Write a projector_config.pbtxt in the logs_path.
    # TensorBoard will read this file during startup.
    embedding_writer = tf.summary.FileWriter(summaries_dir,sess1.graph)
    projector.visualize_embeddings(embedding_writer, config)

    # Run session to evaluate the tensor
    x_test_fc1 = sess1.run(embedding_assign, feed_dict={X: X_test})

    # Save the tensor in model.ckpt file
    saver = tf.train.Saver()
    saver.save(sess1, os.path.join(summaries_dir, "model.ckpt"), global_step)


    # Reshape images from vector to matrix
    x_test_images = np.reshape(np.array(X_test), (-1, img_w, img_h))
    # Reshape labels from one-hot-encode to index
    x_test_labels = np.argmax(Y_test, axis=1)

    write_sprite_image(os.path.join(summaries_dir, 'sprite_images.png'), x_test_images)
    write_metadata(os.path.join(summaries_dir, 'metadata.tsv'), x_test_labels)

Finished Epoch 1
Finished Epoch 2
Finished Epoch 3
Finished Epoch 4
Finished Epoch 5
Finished Epoch 6
Finished Epoch 7
Finished Epoch 8
Finished Epoch 9
Finished Epoch 10
Finished Epoch 11
Finished Epoch 12
Finished Epoch 13
Finished Epoch 14
Finished Epoch 15
Finished Epoch 16
Finished Epoch 17
Finished Epoch 18
Finished Epoch 19
Finished Epoch 20
Sprite image saved in logs/myjupyter16:14:31/sprite_images.png
Metadata file saved in logs/myjupyter16:14:31/metadata.tsv
